In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --buildfile test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@411-297> }
00:00:00 #3 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral\temp\test
00:00:00 #4 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #5 [Debug] runWithTimeoutAsync / timeout: 500
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #7 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:01 #8 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:01 #9 [Verbose] > Server bound to: http://localhost:13805
00:00:01 #10 [Debug] buildFile / takeWhileIn

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.2.2: F# to Rust compiler (status: alpha)

Thanks to the contributor! @davidtme
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 529ms

Started Fable compilation...
Fable compilation finished in 2096ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^
    |
    = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&mut self, item: &Item) {
   |        ^^^^^^^^^^^
   |
   = note: `#[warn(dead_code)]` on by default

   --> apps\spiral\temp\test\./main.rs:124:4
    |
124 | fn parse_comment(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:130:4
    |
130 | fn parse_string(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:145:4
    |
145 | fn parse_identifier(input: &str) -> IResult<&str, Spir

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished release [optimized] target(s) in 25.37s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\test-d4818efb9a1fa553.exe)

running 3 tests
test test_parse_number ... ok
2023-11-04T21:06:09.707754Z  INFO test: input=Operator("+")
2023-11-04T21:06:09.707989Z  INFO test: input=Operator("+")
2023-11-04T21:06:09.708121Z  INFO test: input=Integer(7715133512229082451)
2023-11-04T21:06:09.708361Z  INFO test: input=Operator(")")
2023-11-04T21:06:09.708572Z  INFO test: input=Identifier("pQ3uekmLwfOAaZ8UIeK9ffMvZraTp")
2023-11-04T21:06:09.708653Z  INFO test: input=Operator("*")
2023-11-04T21:06:09.708739Z  INFO test: input=StringLiteral("/,_ZeE^t&H'$E?A-")
2023-11-04T21:06:09.708832Z  INFO test: input=Comment(">{=VwP\"lbu,\"0P17p?!XS")
2023-11-04T21:06:09.708925Z  INFO test: input=StringLiteral("7y&?l&g&7i$x?$}_._+%H#")
2023-11-04T21:06:09.709049Z  INFO test: input=StringLiteral("SsL.9W=6pKs2WjIQm

In [ ]:
{ . $ScriptDir/../../../../target/release/test$(GetExecutableSuffix) } | Invoke-Block

app=test
